# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 10000000 # Set maximum number of steps to run environment.
run_path = "wall_cur" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "Wall" # Name of the training environment file.
curriculum_file = "curricula/wall.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		max_wall_height -> 6.0
		min_wall_height -> 5.5
Unity brain name: Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 16
        Action space type: discrete
        Action space size (per agent): 6
        Memory space size (per agent): 0
        Action descriptions: , , , , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

config = tf.ConfigProto(device_count = {'GPU': 0})
#config.gpu_options.allow_growth = True

with tf.Session(config = config) as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/wall_cur\model-3000000.cptk


INFO:tensorflow:Restoring parameters from ./models/wall_cur\model-3000000.cptk


Saved Model
Step: 3010000. Mean Reward: 0.6224673728012105. Std of Reward: 0.49295293738050217.


INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	max_wall_height -> 5.0, min_wall_height -> 3.5


Step: 3020000. Mean Reward: 0.67937489266701. Std of Reward: 0.40618381961039957.
Step: 3030000. Mean Reward: 0.37878067290665296. Std of Reward: 0.7381834149255789.
Step: 3040000. Mean Reward: 0.42809215177940113. Std of Reward: 0.7122624690945493.
Step: 3050000. Mean Reward: 0.4386863332565105. Std of Reward: 0.7070792496215312.
Saved Model
Step: 3060000. Mean Reward: 0.44561898211829415. Std of Reward: 0.701395327536623.
Step: 3070000. Mean Reward: 0.47824106364428925. Std of Reward: 0.6784790232256227.
Step: 3080000. Mean Reward: 0.4553479645212643. Std of Reward: 0.6898988560435697.
Step: 3090000. Mean Reward: 0.4573401852270158. Std of Reward: 0.6857312976835609.
Step: 3100000. Mean Reward: 0.37375095590109586. Std of Reward: 0.7334533251609332.
Saved Model
Step: 3110000. Mean Reward: 0.34185463010863915. Std of Reward: 0.7536905838592978.
Step: 3120000. Mean Reward: 0.36048167409608695. Std of Reward: 0.7496631550229615.
Step: 3130000. Mean Reward: 0.4379219558062998. Std of Rew

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "c:\users\user\anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-78e447cd20e4>", line 66, in <module>
    trainer.update_model(batch_size, num_epoch)
  File "C:\Users\user\Documents\pj5\ml-agents\python\ppo\trainer.py", line 185, in update_model
    self.model.update_batch], feed_dict=feed_dict)
  File "c:\users\user\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py", line 889, in run
    run_metadata_ptr)
  File "c:\users\user\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py", line 1120, in _run
    feed_dict_tensor, options, run_metadata)
  File "c:\users\user\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py", line 1317, in _do_run
    options, run_metadata)
  File "c:\users\user\anaconda3\envs\tensorflow-gp

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-100000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-100000.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
